In [16]:
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
import os

plotting.setup_mpl()

f1DataCache=os.path.join(os.path.expanduser('~'),'Downloads','F1DataCache')
ff1.Cache.enable_cache(f1DataCache)

In [17]:
import logging
logger=logging.getLogger()
logger.setLevel(logging.WARNING)

In [35]:
weekend=ff1.get_event(2022, 'Bahrain')
fp1=weekend.get_practice(1)
race=weekend.get_race()

In [39]:
fp1.load();
race.load(laps=True);

/home/chris/anaconda3/envs/MachineLearning/lib/python3.10/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/home/chris/anaconda3/envs/MachineLearning/lib/python3.10/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/home/chris/anaconda3/envs/MachineLearning/lib/python3.10/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/home/chris/anaconda3/envs/MachineLearning/lib/python3.10/site-packages/f

In [14]:
import pandas as pd

driverTLA = {}

for driver in fp1.drivers:
    TLA=ff1.api.driver_info(fp1.api_path)[driver]['Tla']
    driverTLA[driver]=TLA
print(driverTLA)

{'1': 'VER', '55': 'SAI', '47': 'MSC', '44': 'HAM', '31': 'OCO', '27': 'HUL', '24': 'ZHO', '23': 'ALB', '22': 'TSU', '20': 'MAG', '18': 'STR', '16': 'LEC', '14': 'ALO', '11': 'PER', '10': 'GAS', '6': 'LAT', '4': 'NOR', '3': 'RIC', '63': 'RUS', '77': 'BOT'}


In [33]:
tst=fp1.laps[fp1.laps['DriverNumber'].isin(['1'])]
tst=tst[tst['IsAccurate']==True]
tst=tst[["LapTime", "Sector1Time", "Sector2Time", "Sector3Time", "Compound", "TyreLife", "Stint", "TrackStatus"]]

In [68]:
tst.shape

(15, 8)

In [57]:
tst.columns

Index(['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound',
       'TyreLife', 'Stint', 'TrackStatus'],
      dtype='object')

In [51]:
raceLaps=race.laps[race.laps["DriverNumber"]=="1"]
raceLaps=raceLaps[raceLaps["IsAccurate"]==True]
raceLaps["LapTime"].mean().total_seconds()

97.907325

In [63]:
import tensorflow as tf
import numpy as np

ConvertCompound={'INTERMEDIATE':-1, 'WET': -2, 'HARD':2, 'MEDIUM':1, 'SOFT':0,}

In [71]:
xData = np.zeros(shape=(tst.shape[0], 8), dtype=float)
yData=np.zeros(shape=(tst.shape[0],1), dtype=float)

idx=0
for __,row in tst.iterrows():
    lap=row["LapTime"].total_seconds()
    s1=row["Sector1Time"].total_seconds()
    s2=row["Sector2Time"].total_seconds()
    s3=row["Sector3Time"].total_seconds()
    cmpd=ConvertCompound[row["Compound"]]
    xData[idx]=np.array([lap,s1,s2,s3,cmpd,row["TyreLife"],row["Stint"],row["TrackStatus"]])
    yData[idx]=np.array([raceLaps["LapTime"].mean().total_seconds()])
    idx+=1
    

In [84]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, input_shape=(8,), activation=None))
model.add(tf.keras.layers.Dense(32, activation=None))
model.add(tf.keras.layers.Dense(1, activation=None))
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(x=xData, y=yData)

1/1 [==============================] - 1s 855ms/step - loss: 126.8014


In [85]:
yData[0]

array([97.907325])

In [86]:
model.predict(xData[0:5])

1/1 [==============================] - 0s 86ms/step


array([[-17.5514  ],
       [-18.026726],
       [-25.125816],
       [-18.298943],
       [-19.967398]], dtype=float32)